# This version will try to learn what the most important words are

import files

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import sent_tokenize
from operator import itemgetter
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import ConfusionMatrixDisplay

load

In [2]:
df = pd.read_csv('../data/df.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

could do more here to explore the neither and both values

move on to language processing

train-test split

In [3]:
X, y = df['text'].to_frame(), df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

add label to X_train for research purposes .. obviously don't include this in the model

reset index to anticipate future problems ... or not reset the index???

In [4]:
X_train.loc[:, 'label'] = [y_train.loc[val] for val in X_train.index]

<ipython-input-4-0cb08c382a27>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'label'] = [y_train.loc[val] for val in X_train.index]


perfunctory exploring should happen here

top ten visualizations for pos. and non-pos.

size of vocabulary

more?

In [5]:
basic_token_pattern = r"(?u)\b\w\w+\b"
tokenizer = RegexpTokenizer(basic_token_pattern)

In [20]:
X_train.loc[:, 'text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)
vocab_raw = set(X_train['text_tokenized'].explode())
print('Size of raw vocabulary:', len(vocab_raw))   

Size of raw vocabulary: 8876


<ipython-input-20-5ba9b3884ac5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, 'text_tokenized'] = X_train['text'].apply(tokenizer.tokenize)


gonna need naive bayes, might not do any other models (markov, etc.)

In [21]:
baseline_model = MultinomialNB()

look at plurality winner to see score to beat

In [22]:
plurality_cv = round(y_train.value_counts(normalize=True)[0],4)
y_train.value_counts(normalize=True)

sentiment
0    0.672366
1    0.327634
Name: proportion, dtype: float64

first model, just ten features

In [23]:
tfidf = TfidfVectorizer(
    max_features = 10
)

X_train_vectorized = tfidf.fit_transform(X_train['text'])

baseline_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:', plurality_cv,
      '\nBaseline: ',baseline_cv)

Plurality: 0.6724 
Baseline:  0.6724


an absolutely miniscule improvement

let's try all words, not just max_features = 10

In [27]:
tfidf = TfidfVectorizer()

X_train_vectorized = tfidf.fit_transform(X_train['text'])

all_words_cv = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

print('Plurality:', plurality_cv,
      '\nBaseline: ', baseline_cv,
      '\nAll Words:', all_words_cv)

Plurality: 0.6724 
Baseline:  0.6724 
All Words: 0.7005


an actual improvement

let's look at which 10 terms were least and most associated with positive sentiment

In [24]:
# instantiate the vectorizer
tfidf = TfidfVectorizer()
# fit the vectorizer on X_train and transform it
X_train_vectorized = tfidf.fit_transform(X_train['text'])

# create array of the word list from this vectorizer with new index
feature_names = np.array(tfidf.get_feature_names())
# create array of the indices of the feature_names array, ordered by tfidf score
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
# create frequency distribution (dictionary) of 1-grams
all_words_freq_dist = FreqDist(X_train['text_tokenized'].explode())

In [44]:
# show the words with the top 10 tfidf values, and their tfidf values
for n in range(-1,-11,-1):
    print(round(X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[n]],4),
          feature_names[sorted_tfidf_index[n]]
         )      

0.9622 worship
0.9278 rocks
0.9184 hmmmm
0.9014 covet
0.8927 orly
0.8842 location
0.8691 charging
0.8686 whoooooo
0.8622 applestore
0.8585 deleting


In [50]:
# show all instances of this word
X_train[X_train['text'].str.contains('worship')]

,text,label,text_tokenized
77,i worship @mention {link} #sxsw,0,"[worship, mention, link, sxsw]"


In [36]:
print('Smallest tfidf:\n{}\n'.format(feature_names[sorted_tfidf_index[:10]]))
print('Largest tfidf: \n{}'.format(feature_names[sorted_tfidf_index[:-11:-1]]))

Smallest tfidf:
['á¾_î¾ð' '_ã' '_ô' 'documents' '¼¼' 'çü' 'öý' 'èï' 'sxsw' 'karaoke']

Largest tfidf: 
['worship' 'rocks' 'hmmmm' 'covet' 'orly' 'location' 'charging' 'whoooooo'
 'applestore' 'deleting']


explore bigrams

let's try n-grams, n from 2 to 7, using all words

In [59]:
def make_ngrams(word_list, n):
    length = len(word_list)
    if length < n:
        return None
    else:
        ngram_list = []
        for i in range(length - n + 1):
            ngram = ''
            for j in range(i, i+n):
                if j > i:
                    ngram += ' '
                ngram += word_list[j]
            ngram_list.append(ngram)
        return ngram_list

In [76]:
n = 4

tfidf = TfidfVectorizer(
    ngram_range = (n,n)
)

X_train_vectorized = tfidf.fit_transform(X_train['text'])

score = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)

feature_names = np.array(tfidf.get_feature_names())

sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()



# create array of the word list from this vectorizer with new index
feature_names = np.array(tfidf.get_feature_names())
# create array of the indices of the feature_names array, ordered by tfidf score
sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()

X_train.loc[:, str(n)+'_grams'] = X_train.loc[:, 'text_tokenized'].apply(lambda x: make_ngrams(x, n))

# create frequency distribution (dictionary) of 1-grams
bigrams_freq_dist = FreqDist(X_train['text_tokenized'].explode())

bigrams = X_train.loc[:, str(n)+'_grams'].explode()
bigrams_freq_dist = FreqDist(bigrams)
    
#     smallest.append(feature_names[sorted_tfidf_index[:10]])
#     largest.append(feature_names[sorted_tfidf_index[:-11:-1]])

<ipython-input-76-cd7379d45510>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.loc[:, str(n)+'_grams'] = X_train.loc[:, 'text_tokenized'].apply(lambda x: make_ngrams(x, n))


In [77]:
# show the words with the top 10 tfidf values, and their tfidf values
for n in range(-1,-11,-1):
    print(round(X_train_vectorized.max(0).toarray()[0][sorted_tfidf_index[n]],4),
          feature_names[sorted_tfidf_index[n]]
         )      

1.0 mention sxsw ipad rocks
1.0 my sxsw iphone screen
1.0 worship mention link sxsw
1.0 essential sxsw tools link
1.0 iphone sharing sxsw shareable
1.0 google circles sxsw orly
0.8038 at apple store at
0.7342 ipad line sxsw link
0.7229 in hand sxsw thisisdare
0.7229 covet new ipad link


In [ ]:
smallest, largest = [], []

for n in range(1,8):
    
    if n > 1:
        tfidf = TfidfVectorizer(
            ngram_range = (n,n)
        )
    else:
        tfidf = TfidfVectorizer()
    
    X_train_vectorized = tfidf.fit_transform(X_train['text'])
    
    score = round(cross_val_score(baseline_model, X_train_vectorized, y_train).mean(),4)
    
    print(str(n)+'-gram', 'score:', score)

    feature_names = np.array(tfidf.get_feature_names())

    sorted_tfidf_index = X_train_vectorized.max(0).toarray()[0].argsort()
    
    smallest.append(feature_names[sorted_tfidf_index[:10]])
    largest.append(feature_names[sorted_tfidf_index[:-11:-1]])

top ten only looks at top ten most frequent, so this is useless until you give it a stop words list, probably not useful at all

now i have a way to find the top and bottom tfidf scores, i need to take this one step further to see how frequent those terms are, this will help me to decide to ignore n > 4 for example

I will make a function to use with lambda to generate a column of ngrams

In [ ]:
def make_ngrams(word_list, n):
    length = len(word_list)
    if length < n:
        return None
    else:
        ngram_list = []
        for i in range(length - n + 1):
            ngram = ''
            for j in range(i, i+n):
                if j > i:
                    ngram += ' '
                ngram += word_list[j]
            ngram_list.append(ngram)
        return ngram_list

In [ ]:
# it might stop throwing an error if you FIRST establish these columns, THEN assign values to them??
for n in range(3,4):
    title = str(n) + '_grams'
    X_train.loc[:, title] = X_train.loc[:, 'text_tokenized'].apply(lambda x: make_ngrams(x, n))

In [ ]:
all_words = X_train.loc[:, '3_grams'].explode()
all_words_freq_dist = FreqDist(all_words)

all_words_set = set(all_words)

all_words_ordered = list(zip(*all_words_freq_dist.most_common(10)))

all_words_ordered

In [ ]:
for item in largest[2]:
    print(all_words_freq_dist[item], item)